# Creating a Simple Agent with Tracing

In [2]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [3]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [4]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out nutrition advice.
    You give concise answers.
    """,
)

Let's execute the Agent:

In [5]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Generally healthy as part of a balanced diet.
    
    Key points:
    - Nutrients: good source of potassium (supports blood pressure), vitamin B6, vitamin C, and fiber.
    - Energy: provides quick, natural energy from carbs; convenient snack.
    - Fiber: supports digestion and satiety.
    
    Per medium banana (~118 g): about 105 calories, 27 g carbs, 14 g sugar, 3 g fiber, ~422 mg potassium, small amounts of vitamin C and B6.
    
    Considerations:
    - If you need to watch potassium (rare kidney issues), or for very low-sugar diets, adjust intake.
    - Riper bananas have more sugar; choose based on your goals.
    - Pair with protein or fat for lasting fullness.
    
    Bottom line: a healthy, convenient choice for most people in moderation.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [6]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Overall, bananas are healthy for most people.

Key points:
- Good sources: potassium, vitamin B6, vitamin C, and fiber.
- About 105 calories per medium banana; ~27 g carbs, ~3 g fiber, ~14% DV potassium.
- Sugar content rises as they ripen; choose firmer for less ripe (less sweet) if needed.
- Pair with protein or fat to help stabilize blood sugar.

Bottom line: a convenient, nutrient-dense fruit that fits most balanced diets, unless you have a specific potassium restriction (e.g., certain kidney conditions).

_Good Job!_